In [1]:
import pandas
import skimage.exposure
import numpy
import mahotas
from aicsimageprocessing import read_ome_zarr

19-Oct-22 21:03:03 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
def add_textures(csvname):

    def measure_3D_textures(full_image,sizes,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        features = {}
        direction_feature_names = """AngularSecondMoment Contrast Correlation Variance
    InverseDifferenceMoment SumAverage SumVariance SumEntropy Entropy
    DifferenceVariance DifferenceEntropy InfoMeas1 InfoMeas2""".split()
        n_directions = 13
        for eachchan in channels.keys():
            image = full_image[0,eachchan,:,:,:]
            pixel_data = skimage.exposure.rescale_intensity(image,out_range='uint8')
            for eachcomp in compartments.keys():
                labels = full_image[0,eachcomp,:,:,:]
                binary_labels = labels>0
                label_data = pixel_data*binary_labels
                for size in sizes:
                    texture_features = mahotas.features.haralick(
                            label_data, distance=size, ignore_zeros=True
                        )
                    for direction, direction_features in enumerate(texture_features):
                        for feature_name, feature in zip(direction_feature_names, direction_features):
                            object_name = "{:}_{:}_{:}_{:02d}_{:02d}".format(compartments[eachcomp],channels[eachchan],feature_name, direction, size)
                            feature_dict[object_name]=feature
        return feature_dict

    sizes = [3,5,10,20]
    feature_dict={}
    direction_feature_names = """AngularSecondMoment Contrast Correlation Variance
    InverseDifferenceMoment SumAverage SumVariance SumEntropy Entropy
    DifferenceVariance DifferenceEntropy InfoMeas1 InfoMeas2""".split()
    directions=13
    channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachchan in channels.keys():
        for eachcomp in compartments.keys():
            for direction in range(13):
                for feature_name in direction_feature_names:
                    for size in sizes:
                        object_name = "Texture_3D_{:}_{:}_{:}_{:02d}_{:02d}".format(compartments[eachcomp],channels[eachchan],feature_name, direction, size)
                        feature_dict[object_name]=[]
    
    df = pandas.read_csv(csvname)
    for eachrow in df['3d_image']:
        im = read_ome_zarr(eachrow).data
        feature_dict = measure_3D_textures(im,[3,5,10,20],feature_dict)

    df.join(pandas.DataFrame(feature_dict))
    df.to_csv(csvname)

                    


In [ ]:
add_textures('../data/mitocells_truncated.csv')